In [1]:
## General imports
import os
import json
import glob
import pandas as pd
import matplotlib.pyplot as plt

## Define locations

In [2]:
locs = {'input_master_data' : os.path.join('..', 'data', 'input', 'data_productie', 'master_data.json'),
        'input_daily_BRU' : os.path.join('..', 'data', 'input', 'data_productie', 'daily_production', 'BRU'),
        'output_folder' : os.path.join('..', 'data', 'output')}
locs

{'input_master_data': '..\\data\\input\\data_productie\\master_data.json',
 'input_daily_BRU': '..\\data\\input\\data_productie\\daily_production\\BRU',
 'output_folder': '..\\data\\output'}

## Read data

In [3]:
with open(locs['input_master_data'], 'r') as f:
    data_raw = json.load(f)
msr_data = pd.json_normalize(data_raw)
msr_data

,BRU msr,STO msr
0,1244,274


In [ ]:
def read_daily(ymd, index, city):
    """
    TODO!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    """


In [4]:
folder_daily = locs['input_daily_BRU']

# import all files which end with .json from folder
json_files_BRU = glob.glob(os.path.join(folder_daily, '*.json'))

json_list = []
for file in json_files_BRU:
    with open(file, 'r') as f:
        data = json.load(f)
    json_list.append(data)

# convert list of dfs to dataframe
df_BRU = pd.json_normalize(json_list)
df_BRU

,DoW,hour,minute,date,maintenance,prod_loss,prod_loss_perc,production
0,Monday,1,0,01-01-2018 00:00:00.0000,No,61,5,1183
1,Tuesday,1,0,01-02-2018 00:00:00.0000,No,106,9,1138
2,Wednesday,1,0,01-03-2018 00:00:00.0000,No,129,10,1115
3,Thursday,1,0,01-04-2018 00:00:00.0000,No,278,22,966
4,Friday,1,0,01-05-2018 00:00:00.0000,No,218,17,1026
...,...,...,...,...,...,...,...,...
1825,Saturday,1,0,12-31-2022 00:00:00.0000,No,203,16,1041
1826,Sunday,1,0,01-01-2023 00:00:00.0000,No,231,19,1013
1827,Monday,1,0,01-02-2023 00:00:00.0000,No,181,15,1063
1828,Tuesday,1,0,01-03-2023 00:00:00.0000,No,198,16,1046


In [5]:
df_BRU[['prod_loss', 'prod_loss_perc', 'production']] = (df_BRU[['prod_loss', 'prod_loss_perc', 'production']]
                                                            .apply(pd.to_numeric, downcast='unsigned', errors='coerce'))
df_BRU[['date','time_ignore']] = df_BRU['date'].str.split(' ', expand = True)
df_BRU = df_BRU.drop(['time_ignore'], axis = 1)

In [6]:
df_BRU.sample(20)

,DoW,hour,minute,date,maintenance,prod_loss,prod_loss_perc,production
1662,Thursday,1,0,07-21-2022,No,75.0,6.0,1169.0
1388,Wednesday,1,0,10-20-2021,No,285.0,23.0,959.0
976,Thursday,1,0,09-03-2020,No,177.0,14.0,1067.0
1066,Wednesday,1,0,12-02-2020,No,140.0,11.0,1104.0
1520,Tuesday,1,0,03-01-2022,No,320.0,26.0,924.0
24,Thursday,1,0,01-25-2018,No,1244.0,100.0,0.0
1695,Tuesday,1,0,08-23-2022,No,220.0,18.0,1024.0
348,Saturday,1,0,12-15-2018,No,1244.0,100.0,0.0
867,Sunday,1,0,05-17-2020,Yes,NaN,NaN,NaN
677,Saturday,1,0,11-09-2019,No,236.0,19.0,1008.0


In [7]:
df_BRU.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1830 entries, 0 to 1829
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   DoW             1830 non-null   object 
 1   hour            1830 non-null   int64  
 2   minute          1830 non-null   int64  
 3   date            1830 non-null   object 
 4   maintenance     1830 non-null   object 
 5   prod_loss       1736 non-null   float64
 6   prod_loss_perc  1736 non-null   float64
 7   production      1736 non-null   float64
dtypes: float64(3), int64(2), object(3)
memory usage: 114.5+ KB


In [8]:
df_BRU.describe()

,hour,minute,prod_loss,prod_loss_perc,production
count,1830.0,1830.0,1736.000000,1736.000000,1736.000000
mean,1.0,0.0,430.288018,34.579493,813.711982
std,0.0,0.0,390.712496,31.413713,390.712496
min,1.0,0.0,51.000000,4.000000,0.000000
25%,1.0,0.0,196.750000,16.000000,860.750000
50%,1.0,0.0,270.500000,22.000000,973.500000
75%,1.0,0.0,383.250000,31.000000,1047.250000
max,1.0,0.0,1244.000000,100.000000,1193.000000
